# Hello World Examples

In this notebook, we will walk you through some Hello World examples in `NVFlare/examples/hello-world` to get familiar with basic workflow of NVIDIA FLARE.

We will run the examples in FLARE system in POC mode using [FLARE API](./flare_api.ipynb). You can also run these examples in FLARE simulator.

Each example is self-contained. You can start from any example. But you must run through the 3 steps of each example in sequence.

Before you can run the examples here, following preparation work must be done,

1. Install a virturalenv following instructions in [README.md](https://github.com/NVIDIA/NVFlare/tree/dev/examples)
2. Install Jupyter Lab and install a new kernel for the virtualenv called `nvflare-example`
3. Install NVFlare following this [notebook](../nvflare_setup.ipynb)
4. Start NVFlare in POC mode following this [notebook](./setup_poc.ipynb). All the examples in this notebook requires 2 clients to run.

## Hello Scatter and Gather

The example job in `hello-world/hello-numpy-sag/jobs/hello-numpy-sag` demonstrate the scatter and gather workflow. See [this](https://nvflare.readthedocs.io/en/main/examples/hello_scatter_and_gather.html#hello-scatter-and-gather) for the details of the example.

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the `hello-numpy-sag` job

In [22]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "../hello-world/hello-numpy-sag/jobs/hello-numpy-sag")
job_id = sess.submit_job(job_folder)

print(f"Job is running with ID {job_id}")

Job is running with ID 7c93d839-6fdc-42ba-96ed-f1db920a9212


### 2. Wait for the job

The command `monitor_job()` will wait for the job till it's done.

In [23]:
sess.monitor_job(job_id)

<MonitorReturnCode.JOB_FINISHED: 0>

### 3. Get the result


In [24]:
import numpy as np
result = sess.download_job_result(job_id)
array = np.load(result + "/workspace/models/server.npy")
print(array)

[[ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]]


## Hello Cross-Site Validation

The example job in `hello-world/hello-numpy-cross/jobs/hello-numpy-cross` demonstrates how to perform cross site validation after training.

Please refer to the [documentation](https://nvflare.readthedocs.io/en/main/examples/hello_cross_val.html) for the details.

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the `hello-numpy-cross-val` job

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "../hello-world/hello-numpy-cross-val/jobs/hello-numpy-cross-val")
job_id = sess.submit_job(job_folder)

print(f"Job is running with ID {job_id}")

### 2. Wait for the job

In [ ]:
sess.monitor_job(job_id)

### 3. Get the result

In [ ]:
import json
import pprint

result = sess.download_job_result(job_id)
with open(result + "/workspace/cross_site_val/cross_val_results.json", "r") as f:
  cross_val_result = json.load(f)

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(cross_val_result)

## Hello Cyclic Weight Transfer

This example uses the CyclicController workflow to implement [Cyclic Weight Transfer](https://pubmed.ncbi.nlm.nih.gov/29617797/) with TensorFlow as the deep learning training framework. The job is `hello-world/hello-cyclic/jobs/hello-cyclic`.

To use this example, tensorflow must be installed using the `requirements.txt`,

    pip install -r hello-world/hello-cyclic/requirements.txt
    
This examples needs access to [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the hello-cyclic job

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "../hello-world/hello-cyclic/jobs/hello-cyclic")
job_id = sess.submit_job(job_folder)
print(f"Job is running with ID {job_id}")

### 2. Wait for the job

In [ ]:
sess.monitor_job(job_id)

### 3. Get the result

In [ ]:
from nvflare.fuel.utils import fobs
from nvflare.app_common.decomposers import common_decomposers
import pprint

# This example stores numpy arrays in FOBS format. Decomposers for Numpy is not registered automatically.
common_decomposers.register()

result = sess.download_job_result(job_id)
with open(result + "/workspace/app_server/tf2weights.fobs", "rb") as f:
    bytes = f.read()

weights = fobs.loads(bytes)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(weights)

## Hello PyTorch

This example demonstrates how to use NVFlare with the popular deep learning framework PyTorch. The job is `hello-world/hello-pt/jobs/hello-pt`.

Refer to the [documentation](https://nvflare.readthedocs.io/en/main/examples/hello_pt.html) for details.

To use this example, PyTorch must be installed using the `requirements.txt`,

    pip install -r hello-world/hello-pt/requirements.txt
    
This examples also needs access to CIFAR10 dataset.

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the hello-pt job

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "../hello-world/hello-pt/jobs/hello-pt")
job_id = sess.submit_job(job_folder)

print(f"Job is running with ID {job_id}")

### 2. Wait for the job

In [ ]:
sess.monitor_job(job_id)

### 3. Get the result

In [ ]:
import os
import pprint
import torch

result = sess.download_job_result(job_id)
model_path = os.path.join(result, "workspace/app_server/FL_global_model.pt")

model = torch.load(model_path)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(model)

## Hello TensorFlow 2

This example demonstrates how to use NVFlare with the popular deep learning framework TensorFlow 2. The job is `examples/hello-world/hello-tf2/jobs/hello-tf2`.

Refer to the [documentation](https://nvflare.readthedocs.io/en/main/examples/hello_tf2.html) for details.

To use this example, PyTorch must be installed using the `requirements.txt`,

    pip install -r hello-world/hello-tf2/requirements.txt
    
This examples also needs access to [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the hello-tf2 job

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.getcwd() + "/../hello-world/hello-tf2/jobs/hello-tf2"
job_id = sess.submit_job(job_folder)
print(f"Job is running with ID {job_id}")

### 2. Wait for the job

In [ ]:
sess.monitor_job(job_id)

### 3. Get the result

In [ ]:
from nvflare.fuel.utils import fobs
from nvflare.app_common.decomposers import common_decomposers
import pprint

common_decomposers.register()
result = sess.download_job_result(job_id)
with open(result + "/workspace/app_server/tf2weights.fobs", "rb") as f:
    bytes = f.read()

weights = fobs.loads(bytes)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(weights)